In [52]:
from pyspark.sql import SparkSession

In [53]:
spark = (
    SparkSession.
    builder.
    config("spark.sql.warehouse.dir", "/home/deepak/programs/python/sparkLearn/spark-warehouse").
    config("javax.jdo.option.ConnectionURL", "jdbc:derby:/home/deepak/programs/python/sparkLearn/metastore_db;create=true").
    config("spark.sql.catalogImplementation", "hive").
    enableHiveSupport().
    appName('SQL').
    getOrCreate()
)

25/10/05 12:24:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [30]:
# 1️⃣ spark.sql.warehouse.dir
# This sets the **location where Spark stores managed table data**.
# Managed tables are created using CREATE TABLE without an explicit LOCATION.
# Example:
#   If you run "CREATE TABLE demo (id INT)", the table's data files
#   will be stored inside this directory.
# warehouse_dir = "/home/deepak/programs/python/sparkLearn/DifferentSources/spark-warehouse"

# 2️⃣ javax.jdo.option.ConnectionURL
# This tells Spark **where the Hive metastore database is located**.
# Spark uses Derby (embedded DB) if no external metastore is configured.
# By pointing it to a path, Spark persists **table metadata** across sessions.
# "create=true" ensures the metastore is created if it doesn't exist.
# metastore_url = "jdbc:derby:/home/deepak/programs/python/sparkLearn/metastore_db;create=true"

# 3️⃣ spark.sql.catalogImplementation
# This specifies the **catalog type Spark will use**.
# Options:
#   - "in-memory": default temporary catalog, disappears when session ends
#   - "hive": uses a Hive metastore to persist table metadata
# Setting this to "hive" ensures tables are persistent across sessions.
# catalog_impl = "hive"


In [26]:
spark.sql("""
show catalogs
""").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [27]:
spark.sql("""
show databases in spark_catalog
""").show()

25/10/05 12:04:36 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/05 12:04:36 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore deepak@127.0.1.1
25/10/05 12:04:36 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException


+---------+
|namespace|
+---------+
|  default|
+---------+



In [28]:
spark.sql("""
show tables in default
""").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [29]:
spark.conf.get("spark.sql.catalogImplementation")

'hive'

In [38]:
spark.sql('drop table flights')

DataFrame[]

In [39]:
spark.sql("""
CREATE TABLE spark_catalog.default.flights (
DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count
LONG)
USING JSON OPTIONS (path '/home/deepak/programs/python/sparkLearn/data/flight-data/json/2015-summary.json')
""")

25/10/05 12:15:01 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider JSON. Persisting data source table `spark_catalog`.`default`.`flights` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [41]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|  flights|      false|
+---------+---------+-----------+



In [48]:
spark.sql('DESCRIBE EXTENDED FLIGHTS').show(truncate=False)

+----------------------------+------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                           |comment|
+----------------------------+------------------------------------------------------------------------------------+-------+
|DEST_COUNTRY_NAME           |string                                                                              |NULL   |
|ORIGIN_COUNTRY_NAME         |string                                                                              |NULL   |
|count                       |bigint                                                                              |NULL   |
|                            |                                                                                    |       |
|# Detailed Table Information|                                                                                    |       |
|Catalog

In [49]:
spark.sql("""
    create table flights_csv (
        DEST_COUNTRY_NAME STRING,
        ORIGIN_COUNTRY_NAME STRING COMMENT "remember, the US will be most prevalent",
        count LONG
    ) using csv options (header true, path '/home/deepak/programs/python/sparkLearn/data/flight-data/csv')
""")

25/10/05 12:22:51 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`default`.`flights_csv` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [50]:
spark.sql('DESCRIBE EXTENDED FLIGHTS_CSV').show(truncate=False)

+----------------------------+-----------------------------------------------------------------+---------------------------------------+
|col_name                    |data_type                                                        |comment                                |
+----------------------------+-----------------------------------------------------------------+---------------------------------------+
|DEST_COUNTRY_NAME           |string                                                           |NULL                                   |
|ORIGIN_COUNTRY_NAME         |string                                                           |remember, the US will be most prevalent|
|count                       |bigint                                                           |NULL                                   |
|                            |                                                                 |                                       |
|# Detailed Table Information|           

In [51]:
spark.sql('select * from flights_csv').show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [54]:
spark.sql("""
    create table managed_table (
        NAME STRING,
        AGE INTEGER
    )
""")

DataFrame[]

In [57]:
spark.sql("""
insert into managed_table values ('deepak', 24)
""")

DataFrame[]

In [59]:
spark.sql('select * from managed_table').show()

+------+---+
|  NAME|AGE|
+------+---+
|deepak| 24|
+------+---+



In [60]:
spark.sql("""
insert into managed_table values ('Krish', 27)
""")

DataFrame[]

In [61]:
spark.sql('select * from managed_table').show()

+------+---+
|  NAME|AGE|
+------+---+
|deepak| 24|
| Krish| 27|
+------+---+



In [ ]:
spark.sql('describe history managed_table').show() # this will not work because the default table it parquet not delta

{"ts": "2025-10-05 12:28:32.118", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `history` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"errorClass": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o106.sql.\n: org.apache.spark.sql.AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `history` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE I

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `history` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01; line 1 pos 9;
'DescribeColumn 'managed_table, false, [info_name#335, info_value#336]
+- 'UnresolvedTableOrView [history], DESCRIBE TABLE, true


In [56]:
spark.sql(
    'DESCRIBE EXTENDED managed_table'
).show(truncate=False)

+----------------------------+-------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                  |comment|
+----------------------------+-------------------------------------------------------------------------------------------+-------+
|NAME                        |string                                                                                     |NULL   |
|AGE                         |int                                                                                        |NULL   |
|                            |                                                                                           |       |
|# Detailed Table Information|                                                                                           |       |
|Catalog                     |spark_catalog                                        

In [ ]:
spark.sql("""
    create table delta_managed_table (
        NAME STRING,
        AGE INTEGER
    ) using csv
""")

25/10/05 15:09:46 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`default`.`delta_managed_table` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [65]:
spark.sql("""
    create table managed_table_csv (
        NAME STRING,
        AGE INTEGER
    ) using csv
""")

25/10/05 15:11:26 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`default`.`managed_table_csv` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [66]:
spark.sql("""
insert into managed_table_csv values ('deepak', 24)
""")

DataFrame[]

In [67]:
spark.sql(
    'DESCRIBE EXTENDED managed_table_csv'
).show(truncate=False)

+----------------------------+-----------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                      |comment|
+----------------------------+-----------------------------------------------------------------------------------------------+-------+
|NAME                        |string                                                                                         |NULL   |
|AGE                         |int                                                                                            |NULL   |
|                            |                                                                                               |       |
|# Detailed Table Information|                                                                                               |       |
|Catalog                     |spark_catalog            

In [23]:
spark.stop()